In [ ]:
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from decimal import Decimal
from benchmark.alphaxutils import *

import requests
import pandas as pd

In [ ]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = '18'
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = str(cgtokens.decimals[i])

In [ ]:
threshold_orders = 6
support_partial = True

# # ETH side orders
# tkn_y = 'ETH'
# tkn_x = 'USDC'

# realisticByTarget_inputAmount = 10 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
# realisticBySource_inputAmount = 30000 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

# order_params = {
#     'liquidity':'5',
#     'highestRate':'0.0005', # 2000
#     'lowestRate':'0.0004',  # 2500
#     'marginalRate':'0.0005',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params2 = {
#     'liquidity':'4',
#     'highestRate':'0.0006', # 1666
#     'lowestRate':'0.0005',  # 2000
#     'marginalRate':'0.0006',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params3 = {
#     'liquidity':Decimal('10'),
#     'highestRate':Decimal('0.0006'), # 1666
#     'lowestRate':Decimal('0.0005'),  # 2000
#     'marginalRate':Decimal('0.0006'),
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }


# USDC side orders
tkn_y = 'USDC'
tkn_x = 'ETH'

realisticByTarget_inputAmount = 30000 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
realisticBySource_inputAmount = 10 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

order_params = {
    'liquidity':'20000',
    'highestRate':'2500',
    'lowestRate':'2000',
    'marginalRate':'2500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params2 = {
    'liquidity':'17000',
    'highestRate':'3500',
    'lowestRate':'3000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params3 = {
    'liquidity':'15000',
    'highestRate':'3500',
    'lowestRate':'2000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }

orders = []
for i in range(1):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]
    orders += [Order(order_params3)]

In [ ]:
print("inputAmount",realisticByTarget_inputAmount)
TargetActions2 = mpr_matchByTarget(realisticByTarget_inputAmount, orders, threshold_orders, support_partial)
total_output_Target2 = sum([TargetActions2[k]['dx'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_input_Target2 = sum([TargetActions2[k]['dy_specified'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Target2)
print("Total Return Real:", total_output_Target2)
print("Rate Real:",(total_input_Target2/total_output_Target2))
TargetActions2

In [ ]:
print("inputAmount",realisticBySource_inputAmount)
SourceActions2 = mpr_matchBySource(realisticBySource_inputAmount, orders, threshold_orders, support_partial)
total_input_Source2 = sum([SourceActions2[k]['dx_specified'] for k in SourceActions2.keys()])  / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_output_Source2 = sum([SourceActions2[k]['dy'] for k in SourceActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Source2)
print("Total Return:", total_output_Source2)
print("Rate:",total_output_Source2/total_input_Source2)
SourceActions2

In [3]:
test = {
    "amount": "41040000000000000000",
    "orders": [
        {
            "liquidity": "123000000000000000000",
            "lowestRate": "950000000",
            "highestRate": "1050000000",
            "marginalRate": "1000000000"
        },
        {
            "liquidity": "456000000000000000000",
            "lowestRate": "950000000",
            "highestRate": "1050000000",
            "marginalRate": "1000000000"
        },
        {
            "liquidity": "789000000000000000000",
            "lowestRate": "950000000",
            "highestRate": "1050000000",
            "marginalRate": "1000000000"
        },
        {
            "liquidity": "123000000000000000000",
            "lowestRate": "760000000",
            "highestRate": "840000000",
            "marginalRate": "800000000"
        },
        {
            "liquidity": "456000000000000000000",
            "lowestRate": "760000000",
            "highestRate": "840000000",
            "marginalRate": "800000000"
        },
        {
            "liquidity": "789000000000000000000",
            "lowestRate": "760000000",
            "highestRate": "840000000",
            "marginalRate": "800000000"
        },
        {
            "liquidity": "123000000000000000000",
            "lowestRate": "633333333.333333333333",
            "highestRate": "700000000",
            "marginalRate": "666666666.666666666667"
        },
        {
            "liquidity": "456000000000000000000",
            "lowestRate": "633333333.333333333333",
            "highestRate": "700000000",
            "marginalRate": "666666666.666666666667"
        },
        {
            "liquidity": "789000000000000000000",
            "lowestRate": "633333333.333333333333",
            "highestRate": "700000000",
            "marginalRate": "666666666.666666666667"
        }
    ],
    "oldMethodOutputActions": [
        {
            "id": "2",
            "input": "41040000000000000000",
            "output": "41094123201"
        }
    ]
}

method = mpr_matchByTarget

amount = int(test['amount'])

orders = [Order({**order_params, 'dec_y': 0, 'dec_x': 0}) for order_params in test['orders']]

actions = mpr_matchByTarget(amount, orders, 1, True)

print(actions)

{2: {'dy_specified': 41040000000000000022, 'dx': 41094123201}}


AssertionError: 

In [2]:
from benchmark import util
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from benchmark.alphaxutils import Order

for fileName in ['ArbitraryMatch', 'BigPoolMatch', 'EthUsdcMatch']:
    for test in util.read('', f'benchmark/resources/match/{fileName}'):
        if any(order_params['lowestRate'] == order_params['highestRate'] for order_params in test['orders']):
            continue
        method = globals()['mpr_' + test['method'].split('Amount')[0]]
        amount = int(test['amount'])
        orders = [Order({**order_params, 'dec_y': 0, 'dec_x': 0}) for order_params in test['orders']]
        oldActions = test['actions']
        threshold = len(oldActions)
        while True:
            newActions = method(amount, orders, threshold, True)
            if test['method'] == 'matchBySourceAmount':
                oldInputSum  = sum(int(action['input'       ]) for action in oldActions)
                newInputSum  = sum(int(action['dx_specified']) for action in newActions.values())
                oldOutputSum = sum(int(action['output'      ]) for action in oldActions)
                newOutputSum = sum(int(action['dy'          ]) for action in newActions.values())
                if not (newOutputSum+1 >= oldOutputSum):
                    threshold += 1
                    print(threshold - len(oldActions), threshold, len(oldActions))
                    if threshold - len(oldActions) < 10:
                        continue
                    print(util.dumps(test, indent = 4))
                    for key in newActions:
                        obj = {
                            'id': key,
                            'input': str(newActions[key]['dx_specified']),
                            'output': str(newActions[key]['dy']),
                        }
                        print(util.dumps(obj, indent = 4))
                    print('oldInputSum  =', oldInputSum )
                    print('newInputSum  =', newInputSum )
                    print('oldOutputSum =', oldOutputSum)
                    print('newOutputSum =', newOutputSum)
                    assert False
            if test['method'] == 'matchByTargetAmount':
                oldInputSum  = sum(int(action['input'       ]) for action in oldActions)
                newInputSum  = sum(int(action['dy_specified']) for action in newActions.values())
                oldOutputSum = sum(int(action['output'      ]) for action in oldActions)
                newOutputSum = sum(int(action['dx'          ]) for action in newActions.values())
                if not (newOutputSum <= oldOutputSum+1):
                    threshold += 1
                    print(threshold - len(oldActions), threshold, len(oldActions))
                    if threshold - len(oldActions) < 10:
                        continue
                    print(util.dumps(test, indent = 4))
                    for key in newActions:
                        obj = {
                            'id': key,
                            'input': str(newActions[key]['dy_specified']),
                            'output': str(newActions[key]['dx']),
                        }
                        print(util.dumps(obj, indent = 4))
                    print('oldInputSum  =', oldInputSum )
                    print('newInputSum  =', newInputSum )
                    print('oldOutputSum =', oldOutputSum)
                    print('newOutputSum =', newOutputSum)
                    assert False
            break

{
    "method": "matchBySourceAmount",
    "amount": "1000000",
    "orders": [
        {
            "liquidity": "2962962",
            "lowestRate": "1",
            "highestRate": "2",
            "marginalRate": "2"
        },
        {
            "liquidity": "3950616",
            "lowestRate": "2",
            "highestRate": "3",
            "marginalRate": "3"
        },
        {
            "liquidity": "4938270",
            "lowestRate": "3",
            "highestRate": "4",
            "marginalRate": "4"
        },
        {
            "liquidity": "5925924",
            "lowestRate": "4",
            "highestRate": "5",
            "marginalRate": "5"
        },
        {
            "liquidity": "6913578",
            "lowestRate": "5",
            "highestRate": "6",
            "marginalRate": "6"
        },
        {
            "liquidity": "7901232",
            "lowestRate": "6",
            "highestRate": "7",
            "marginalRate": "7"
        },
        

AssertionError: 